In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
#from delta import *

# Inicializar a sessão Spark
minio_endpoint = "http://127.0.0.1:9000"
minio_access_key = "minio"
minio_secret_key = "minio123"

spark = SparkSession.builder \
    .appName("test13") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

print("Sessão Spark inicializada com suporte ao Delta Lake.")

delta_table_path = "s3a://raw/ecommerce_db/postgres.public.products*.json"
products_df = spark.read.format("json").load(delta_table_path)

Sessão Spark inicializada com suporte ao Delta Lake.
Dataframe criado:
+-------+------------+-------+
|   nome|departamento|salario|
+-------+------------+-------+
|  Alice|      Vendas|   5000|
|    Bob|          TI|   6000|
|Charlie|      Vendas|   4500|
|  David|   Marketing|   4000|
|    Eve|          TI|   5500|
+-------+------------+-------+



In [18]:
delta_table_path = "s3a://raw/ecommerce_db/postgres.public.products*.json"
products_df = spark.read.format("json").load(delta_table_path)

In [6]:
delta_table_path = "s3a://raw/xd.json"
df.write.format("delta").save(delta_table_path)

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 40836)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [3]:
spark.stop()

In [3]:
!python --version

Python 3.8.13
